# 'Utilities' needed, to be moved to own file with time

In [5]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from sklearn import gaussian_process
from sklearn.gaussian_process.kernels import RBF
from matplotlib import interactive
import numpy as np
import scipy.stats
from numpy.linalg import inv
import matplotlib.pyplot as plt

def plot_gp(mu, cov, X, X_train = None, y_train = None, samples = []):
    X = X.ravel()
    mu = mu.ravel()
    conf_interval = 1.96 * np.sqrt(np.diag(cov))

    plt.fill_between(X, mu + conf_interval, mu - conf_interval, alpha = 0.3)
    for i, sample in enumerate(samples):
        plt.plot(X, sample, lw = 1, ls = ':', label = f'Sample {i + 1}')

    if X_train is not None:
        plt.plot(X_train[0:-1], y_train[0:-1], 'kx')
        plt.plot(X_train[-1], y_train[-1], 'rx')
        plt.plot(X, mu, lw = 1, label = 'Mean function', color = 'black')
    plt.ylim(-1, 5)
    plt.legend()
    plt.show()

def plot_acq_function(X, mu, stdevs, y_max, acq_func, **params):
    acq_values = acq_func(mu, stdevs, y_max, **params)
    plt.plot(X, acq_values[-1], lw = 1.5, label = 'Acquisition function value')
    plt.show()

    
# EXAMPLE FUNCTIONS
def example_func(X_val):
    return np.exp(np.sin(np.power(X_val - 4 , 2)) + 0.5)

# EXAMPLE FUNCTIONS
def example_func2(X_val):
    return (-1) *(np.sin(np.power(X_val - 1 , 2)) + np.exp(X_val/3.1) - X_val/2) + 4


def example_2d(X1, X2):
    return np.array(np.sin(np.sqrt(X1 ** 2 + X2 ** 2)) + X1/5).reshape(-1, 1)


# ACQUISITION FUNCTIONS
# takes one point and evaluates the acq.function at said point
def expected_imp(mu_n, std_n, y_max, epsilon = 1):
    norm_y = (mu_n - y_max - epsilon) / std_n
    return (-1) * (norm_y * std_n * scipy.stats.norm.pdf(norm_y) + std_n * scipy.stats.norm.cdf(norm_y))

def prob_imp(mu_n, std_n, y_max, epsilon = 1):
    norm_y = (mu_n - y_max - epsilon)/std_n
    return scipy.stats.norm.cdf(norm_y)

def UCB(mu_n, std_n, y_max, beta = 20):
    return mu_n + std_n * beta


# KERNELS
# X1 - Point in n-dim room for which cov. is to be computed, m * d
# X2 - points already computed, n * d
# computing covariance matrix between X and X_other
def squared_exp(X1, X2, l = 1.0, sigma_f = 1.0): 
    
    sqdist = np.sum(np.power(X1, 2), 1).reshape(-1, 1) + np.sum(np.power(X2, 2), 1) - 2 * np.dot(X1, X2.T)
    return np.power(sigma_f, 2) * np.exp((-0.5)/np.power(l, 2) * sqdist)

def matern52(X1, X2, sigma_f = 1.0, sigma_l = 1.0):
    sqdist = np.sum(np.power(X1, 2), 1).reshape(-1, 1) + np.sum(np.power(X2, 2), 1) - 2 * np.dot(X1, X2.T)
    
    return np.power(sigma_f, 2) * (1 + np.sqrt(5) * sqdist/ sigma_l + np.power((np.sqrt(5) * sqdist/ sigma_l), 2)
                                   ) * np.exp((-1)*np.sqrt(5) * sqdist/sigma_l)



# Bayesian Optimization functions, may be moved to separate

In [6]:
def compute_posterior(X, X_train, y_train, kernel):

    K = kernel(X_train, X_train) + 1e-10 *np.eye(len(X_train))
    k_n = kernel(X, X_train)

    # sample length * sample length in size
    k_nn = kernel(X, X) + 1e-10 * np.eye(len(X))
    K_inv = inv(K)
    mu = k_n.dot(K_inv).dot(y_train)
    cov = k_nn - k_n.dot(K_inv).dot(k_n.T)
    print('K.shape', K.shape, '\n')
    print('k_n.shape', k_n.shape, '\n')
    print('k_nn.shape', k_nn.shape, '\n')
    print('K_inv.shape', K_inv.shape, '\n')
    
    
    return mu, cov

def query_random_points(no_points, X, objective):
    points = np.random.randint(0, X.shape[0] - 1, size = (no_points, X.shape[1]))
    X_train = np.array([[X[points[i][dim]][dim] for dim in range(X.shape[1])] for i in range(no_points)])
    y_train =  objective(X_train[:,0], X_train[:,1])
    return X_train, y_train

def find_next_point(y_max, mu, cov, acq_func):
    acq_values = acq_func(mu, cov, y_max)
    X_next = np.argmax(acq_values)
    return X_next

In [8]:
def bayesian_optimization(objective, no_initial, no_iters, X_range, kernel = squared_exp, acq_func = UCB):
# Creating function and parameters to be optimized
    # objective - known or unknown function to be optimized
    # params - list of parameter names and their associated ranges in dict form

    # Creates a proper grid for x for the input range, 4001 samples (must be column vector) - WORKS
    X = np.array([np.linspace(X_range[key][0], X_range[key][1], 1001) for key in X_range.keys()]).T 
    mu = np.zeros(X.shape)
    cov = squared_exp(X, X)
    
    print(X.shape, mu.shape, cov.shape)
    # query_next_point FOR a number of random points equal to no_initial - WORKS
    X_train, y_train = query_random_points(no_initial, X, objective)

    # compute_posterior when done - for range of X, training samples (X_train, y_train)
    mu, cov = compute_posterior(X, X_train, y_train, kernel)
    print(mu.shape, cov.shape)
    #plot_gp(mu, cov, X, X_train = X_train, y_train = y_train)
    
    for iter in range(no_iters):
        y_max = y_train.max()
        std = (np.sqrt(np.diag(cov)+ 1e-10 * np.ones(cov.shape[0])).reshape(-1, 1)) 
        # Find the next point to evaluate, X_next - WORKS
        X_next = X[find_next_point(y_max, mu, std, acq_func)]
        X_train, y_train = np.append(X_train, X_next).reshape(-1, 1), np.append(y_train, objective(X_next)).reshape(-1, 1)
        print('Evaluating: ', X_next)
    
        mu, cov = compute_posterior(X, X_train, y_train, kernel)
        
        plot_gp(mu, cov, X, X_train = X_train, y_train = y_train)
            
bayesian_optimization(example_2d, 5, 30, {'X1':[-4, 4], 'X2':[-4, 4]})

[32.       31.872128 31.744512 ... 31.744512 31.872128 32.      ]
(1001, 2) (1001, 2) (1001, 1001)
[12.56512  14.32832  17.7616    6.106432  2.277952]
[32.       31.872128 31.744512 ... 31.744512 31.872128 32.      ]
[32.       31.872128 31.744512 ... 31.744512 31.872128 32.      ]
K.shape (5, 5) 

k_n.shape (1001, 5) 

k_nn.shape (1001, 1001) 

K_inv.shape (5, 5) 

(1001, 1) (1001, 1001)


TypeError: example_2d() missing 1 required positional argument: 'X2'

In [9]:
X = np.array([0, 1,6,4,7,-44])
X.size
X1_t = np.linspace(-4, 4, 1001)
X2_t = np.linspace(-4, 4, 1001)
X1, X2 = np.meshgrid(X1_t, X2_t)
y = example_2d(X1, X2)

%matplotlib qt

fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.plot_wireframe(X1, X2, y, rstride = 30, cstride = 30)
ax.plot_wireframe(X1, X2, y, rstride = 30, cstride = 30)
interactive(True)
plt.show()

ValueError: shape mismatch: objects cannot be broadcast to a single shape